<div class="list-group" id="list-tab" role="tablist">
<h1 class="list-group-item list-group-item-action active" data-toggle="list" style='background:maroon; border:0; color:yellow' role="tab" aria-controls="home"><center>Covid19 India EDA + Forecasting</center></h1>


![covid](https://hms.harvard.edu/sites/default/files/hero-images/coronavirus.jpg)

In [ ]:
import IPython

url = "https://app.flourish.studio/visualisation/1813946/"
iframe = '<iframe src=' + url + ' width=820 height=350></iframe>'
IPython.display.HTML(iframe)

# <font color=darkred>`If you found this notebook helpful , some upvotes would be very much appreciated - That will keep me motivated :)`


![](http://)![](http://)## <font color=darkgreen>Approach:
- <i><b>Data cleaning and statistical analysis.
- Exploratory Data Analysis and visualisations.
- Forecasting using FBProphet
- Forecasting using ARIMA

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_context("paper", font_scale = 1, rc={"grid.linewidth": 3})
pd.set_option('display.max_rows', 100, 'display.max_columns', 400)
import geopandas as gpd
from fbprophet import Prophet
from fbprophet.plot import plot_plotly
import datetime

In [ ]:
stats = pd.read_csv('../input/state-wise-dataset/State/states.csv')
stats=stats.drop(['Tested','Other'],axis=1)
stats['Date']=pd.to_datetime(stats['Date'],format='%Y-%m-%d')


In [ ]:
today=stats[stats.Date == '2020-11-01']
shp = gpd.read_file('../input/india-shape-polygon/Shape/india-polygon.shp')
merged = shp.merge(today ,  left_on='st_nm', right_on='State')

In [ ]:
merged.head()

In [ ]:
merged['coords'] = merged['geometry'].apply(lambda x: x.representative_point().coords[:])
merged['coords'] = [coords[0] for coords in merged['coords']]


sns.set_context("talk")
sns.set_style("dark")
cmap = 'YlGn'
figsize = (25, 20)
ax = merged.plot(column= 'Confirmed', cmap=cmap, 
                          figsize=figsize, scheme='User_Defined',
                          classification_kwds=dict(bins=[100,1000,10000,50000,100000,200000,1000000]),
                          edgecolor='black', legend = True)

for idx, row in merged.iterrows():
    ax.text(row.coords[0], row.coords[1], s=row['Confirmed'], 
           horizontalalignment='center', bbox={'facecolor': 'yellow', 
                                               'alpha':0.8, 'pad': 5, 'edgecolor':'blue'})

ax.get_legend().set_bbox_to_anchor((0.66,0.9))
ax.get_legend().set_title('Number of cases')
ax.set_title("Confirmed Cases of Covid - 19 ", size = 25)
leg = ax.get_legend()


ax.set_axis_off()
plt.axis('equal')
plt.show()

In [ ]:
sns.set_context("talk")
sns.set_style("dark")
cmap = 'OrRd'
figsize = (25, 20)
ax = merged.plot(column= 'Deceased', cmap=cmap, 
                          figsize=figsize, scheme='User_Defined',
                          classification_kwds=dict(bins=[10,50,100,500,1000,2000,4000,8000]),
                          edgecolor='black', legend = True)

for idx, row in merged.iterrows():
    ax.text(row.coords[0], row.coords[1], s=row['Deceased'], 
           horizontalalignment='center', bbox={'facecolor': 'yellow', 
                                               'alpha':0.8, 'pad': 2, 'edgecolor':'black'})

ax.get_legend().set_bbox_to_anchor((0.66,0.9))
ax.get_legend().set_title('Number of deaths')
ax.set_title("Deceased due to Covid -19 ", size = 25)
leg = ax.get_legend()
ax.set_axis_off()
plt.axis('equal')
plt.show()

In [ ]:
sns.set_context("talk")
sns.set_style("dark")
cmap = 'Blues'
figsize = (25, 20)
ax = merged.plot(column= 'Recovered', cmap=cmap, 
                          figsize=figsize, scheme='User_Defined',
                          classification_kwds=dict(bins=[100,1000,10000,50000,100000]),
                          edgecolor='black', legend = True)

for idx, row in merged.iterrows():
    ax.text(row.coords[0], row.coords[1], s=row['Recovered'], 
           horizontalalignment='center', bbox={'facecolor': 'yellow', 
                                               'alpha':0.8, 'pad': 2, 'edgecolor':'black'})

ax.get_legend().set_bbox_to_anchor((0.66,0.9))
ax.get_legend().set_title('Number of Recovered')
ax.set_title("Recovered Cases of Covid-19", size = 25)
leg = ax.get_legend()

ax.set_axis_off()
plt.axis('equal')
plt.show()

In [ ]:
merged['dcratio'] = merged['Deceased'] / merged['Recovered'] * 100
merged.dcratio = merged.dcratio.round(2)
sns.set_context("talk")
sns.set_style("dark")
cmap = 'OrRd'
figsize = (25, 20)
ax = merged.plot(column= 'dcratio', cmap=cmap, 
                          figsize=figsize, scheme='User_Defined',
                          classification_kwds=dict(bins=[0.1 , 0.2 , 0.4 , 0.8 , 1.6 , 3.2 , 6.4]),
                          
                          edgecolor='black', legend = True)

for idx, row in merged.iterrows():
    ax.text(row.coords[0], row.coords[1], s=row['dcratio'], 
           horizontalalignment='center', bbox={'facecolor': 'azure', 
                                               'alpha':0.8, 'pad': 2, 'edgecolor':'black'})

ax.get_legend().set_bbox_to_anchor((0.66,0.9))
ax.get_legend().set_title('Percentage')
ax.set_title("Fatality Rate Per State", size = 25)

ax.set_axis_off()
plt.axis('equal')
plt.show()

In [ ]:
#raw1= pd.read_csv('dataset/raw_data1.csv')
#raw1.drop(labels=['Estimated Onset Date','Patient Number','State Patient Number','Backup Notes'],axis=1,inplace=True)
#raw1.rename(columns={'Num cases':'Num Cases'},inplace=True)
#raw2 = pd.read_csv('dataset/raw_data2.csv')
#raw2.drop(labels=['Estimated Onset Date','Patient Number','State Patient Number','Backup Notes'],axis=1,inplace=True)
#raw2.rename(columns={'Num cases':'Num Cases'},inplace=True)
#raw3 = pd.read_csv('dataset/raw_data3.csv')
#raw3.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw4 = pd.read_csv('dataset/raw_data4.csv')
#raw4.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)

In [ ]:
#raw5 = pd.read_csv('dataset/raw_data5.csv')
#raw5.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw6 = pd.read_csv('dataset/raw_data6.csv')
#raw6.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw7 = pd.read_csv('dataset/raw_data7.csv')
#raw7.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw8 = pd.read_csv('dataset/raw_data8.csv')
#raw8.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw9 = pd.read_csv('dataset/raw_data9.csv')
#raw9.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw10 = pd.read_csv('dataset/raw_data10.csv')
#raw10.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw11 = pd.read_csv('dataset/raw_data11.csv')
#raw11.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw12 = pd.read_csv('dataset/raw_data12.csv')
#raw12.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw13 = pd.read_csv('dataset/raw_data13.csv')
#raw13.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw14 = pd.read_csv('dataset/raw_data14.csv')
#raw14.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw15 = pd.read_csv('dataset/raw_data15.csv')
#raw15.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)
#raw16 = pd.read_csv('dataset/raw_data16.csv')
#raw16.drop(labels=['Entry_ID','State Patient Number','Patient Number'],axis=1,inplace=True)

In [ ]:
#raw = pd.concat([raw1,raw2,raw3,raw4,raw5,raw6,raw7,raw8,raw9,raw10,raw11,raw12,raw13,raw14,raw15,raw16])
#raw.to_csv('raw.csv')

In [ ]:
raw=pd.read_csv('../input/covid19-raw-merged-dataset/raw.csv')
raw.info()

In [ ]:
raw_df = raw[['Age Bracket','Detected City','Detected State','State code','Current Status','Date Announced','Gender',\
    'Num Cases']]
raw_df.head()

## _Age Dataframe_

In [ ]:
age_df= raw_df[['Age Bracket','Current Status','Gender','Num Cases']]
age_df.head()

## _Recovered cases percentage Age wise_

In [ ]:
age_df1 = age_df[~age_df['Age Bracket'].isnull()]
age_df1=age_df1[~(age_df1['Current Status']=='Migrated_Other')]
age_df1.head()

In [ ]:
print(len(age_df1))
print(age_df1['Current Status'].unique())
print(age_df1['Age Bracket'].unique())
print((age_df1['Age Bracket']=='18-28').sum())


In [ ]:
age_df1=age_df1[:-2]
print(len(age_df1))
print(age_df1.info())

In [ ]:
age_df1=age_df1[~age_df1['Gender'].isnull()]
len(age_df1)

In [ ]:
print((age_df1['Age Bracket']=='1 DAY').sum())
print((age_df1['Age Bracket']=='5 months').sum())
print((age_df1['Age Bracket']=='5 Months').sum())
print((age_df1['Age Bracket']=='9 Months').sum())
print((age_df1['Age Bracket']=='8 month').sum())
print((age_df1['Age Bracket']=='6 Months').sum())



In [ ]:
age_df1=age_df1[~age_df1['Age Bracket'].isin(['1 DAY','5 months','5 Months',\
                                              '9 Months','9 Month','8 month','6 Months','28-35','18-28','16 DAYS',\
                                             '2 Months','4 Months','1.5 Months'])]

In [ ]:
len(age_df1)

In [ ]:
age_df1['Age Bracket']=age_df1['Age Bracket'].astype('float32').astype('int32')

In [ ]:
bins = [0,21, 41, 61, 81,120]
labels = ['0-20', '20-40', '40-60', '60-80', '80+']
age_df1['Age range'] = pd.cut(age_df1['Age Bracket'], bins, labels = labels,include_lowest = True)

In [ ]:
age_df1=age_df1[~(age_df1['Gender']=='Non-Binary')]

In [ ]:
age_df1['Gender'].replace({'M ':'M'},inplace=True)

In [ ]:
age_hosp=age_df1[age_df1['Current Status']=='Hospitalized']

In [ ]:
plt.figure(figsize=(10,8))
ax = sns.barplot(data=age_hosp,x='Age range',y='Num Cases',estimator=lambda x:(sum(x)/age_hosp['Num Cases'].sum())*100,hue='Gender')
plt.title('Percentage of cases Age wise with Gender',fontdict={'family': 'serif','color':'black','weight': 'heavy',\
                                                   'size': 14})
plt.ylabel('Percentage of cases',fontsize=14)
plt.xlabel('Age Group',fontsize=14)
# remove all the ticks (both axes), and tick labels on the Y axis
plt.tick_params(axis='both',top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom='on')
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2,height+0.6,'{:1.2f}'.format(height),ha="center",fontsize=10,fontweight='bold',\
            alpha=0.8) 

    # remove the frame of the chart
for spine in plt.gca().spines.values():
    spine.set_visible(False)
plt.show()

## *Percentage of patients recovered age wise*

In [ ]:
age_recvr = age_df1[age_df1['Current Status']=='Recovered']
age_recvr.head()

In [ ]:
age_recvr_df = (age_recvr.groupby('Age range')['Num Cases'].sum()/age_recvr['Num Cases'].sum())*100
age_recvr_df

In [ ]:
plt.figure(figsize=(8,8))
colors = {labels[0]: '#ff99ff', labels[1]: '#ff9999', labels[2]: '#66b3ff', labels[3]:'#ffcc99',labels[4]:'#99ff99'}
handles = [plt.Rectangle((0,0),1,1, color=colors[i]) for i in labels]
explode = (0.01, 0.01, 0.01, 0.01,0.01)
plt.pie(age_recvr_df.values,autopct='%.0f%%',explode=explode,colors=colors.values())
plt.legend(handles,labels)
plt.title('Percentage of Recovered Cases Age wise',fontdict={'family': 'serif','weight': 'bold',\
                                                   'size': 13})
plt.show()

## _Percentage of patients deceased age wise_

In [ ]:
age_dec = age_df1[age_df1['Current Status']=='Deceased']
age_dec['Gender']=age_dec['Gender'].replace({'Femal e':'F','M,':'M'})
age_dec.head()

In [ ]:
age_dec['Gender'].unique()

In [ ]:
len(age_dec)

In [ ]:
age_dec_df = (age_dec.groupby('Age range')['Num Cases'].sum()/age_dec['Num Cases'].sum())*100
age_dec_df

In [ ]:
age_dec_gender = (age_dec.groupby(['Age range','Gender'])['Num Cases'].sum()/age_dec['Num Cases'].sum())*100
age_dec_gender

In [ ]:
plt.figure(figsize=(8,8))
colors = {labels[0]: '#ff99ff', labels[1]: '#ff9999', labels[2]: '#66b3ff', labels[3]:'#ffcc99',labels[4]:'#99ff99'}
handles = [plt.Rectangle((0,0),1,1, color=colors[i]) for i in labels]
explode = (0.01, 0.01, 0.01, 0.01,0.01)
plt.pie(age_dec_df.values,autopct='%.0f%%',explode=explode,colors=colors.values())
plt.legend(handles,labels)
plt.title('Percentage of Deceased Cases Age wise',fontdict={'family': 'serif','weight': 'bold',\
                                                   'size': 13})
plt.show()

In [ ]:
age_dec_df.values

In [ ]:
from matplotlib.legend import Legend
plt.figure(figsize=(8,8))
 
# Data to plot
labels=sorted(list(age_dec['Age range'].unique()))

age_m_f= age_dec_df.values
labels_gender = ['F','M','F','M','F','M','F','M','F','M']
gender_size=age_dec_gender.values
colors = ['#ff6666', '#ffcc99', '#99ff99', '#66b3ff','#ffff66']
explode = (0.02,0.02,0.02,0.02,0.02) 
explode_gender = (0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02,0.02)
colors_gender = ['#c2c2f0','#66ffff', '#c2c2f0','#66ffff', '#c2c2f0','#66ffff', '#c2c2f0','#66ffff','#c2c2f0',\
                 '#66ffff']
handles_1 = [plt.Rectangle((0,0),1,1, color=colors_gender[i]) for i in range(0,len(colors_gender[:2]))]

slices1, legendlabels1, pctlabels1=plt.pie(age_m_f, colors=colors,explode=explode,startangle=90,frame=True,autopct='%.0f%%',pctdistance=0.9,\
                                          textprops={'fontweight':'bold'})
slices2, legendlabels2, pctlabels2=plt.pie(gender_size,explode=explode_gender,colors=colors_gender,radius=0.75,startangle=90,pctdistance=0.8,\
            autopct='%.0f%%',textprops={'fontsize':8})
centre_circle = plt.Circle((0,0),0.5,color='black', fc='white',linewidth=0)
plt.legend(slices1,labels,loc='upper left')
plt.title('Deceased Percentage of Males and Females Age wise',fontdict={'family': 'serif','weight': 'bold',\
                                                   'size': 13})
fig = plt.gcf()
second = Legend(fig,handles_1,['F','M'],loc='upper right',frameon=True)
fig.gca().add_artist(centre_circle)
fig.gca().add_artist(second)
plt.axis('equal')
plt.tight_layout()
plt.show()



#### <font color=darkgreen>_Deceased percentage of males are greater than females. Deceased percebtage in age group 40-60 and 60-80 is highest_

## _ICMR Testing data EDA_

In [ ]:
icmr= pd.read_csv('../input/icmr-dataset/tested_numbers_icmr_data.csv')
icmr.info()

In [ ]:
icmr = icmr[['Tested As Of', 'Total Samples Tested','Total Individuals Tested',\
             'Total Positive Cases','Test positivity rate','Tests Per Confirmed Case', 'Tests per million']]

In [ ]:
icmr['Test positivity rate']=icmr['Test positivity rate'].\
astype('str').apply(lambda x: x.split('%')[0]).astype('float64')

In [ ]:
icmr['Total Positive Cases'].replace({'12,581':12581},inplace=True)
icmr['Total Positive Cases']=icmr['Total Positive Cases'].astype('float64')

In [ ]:
icmr['Total Positive Cases']=icmr['Total Positive Cases'].\
fillna(icmr['Total Samples Tested']*icmr['Test positivity rate']/100)

In [ ]:
icmr=icmr[icmr['Total Samples Tested'].notna()]


In [ ]:
icmr['Tested As Of']=pd.to_datetime(icmr['Tested As Of'],format='%d/%m/%Y').sort_values()

In [ ]:
icmr = icmr.groupby('Tested As Of').sum()
icmr.reset_index(inplace=True)
icmr.sort_values('Tested As Of',inplace=True)

In [ ]:
icmr['Tested As Of']=icmr['Tested As Of'].astype('object').astype('str').apply(lambda x:x.split(' ')[0])
icmr['Tested As Of']

In [ ]:
import matplotlib.dates as md
plt.figure(figsize=(20,15))
bar = sns.barplot(data=icmr[::3],x='Tested As Of',y='Total Samples Tested')
bar.set_xticklabels(bar.get_xticklabels(), rotation=45, horizontalalignment='right',fontsize=10.5)
plt.title('Times Series plot for No of Samples tested in India',fontdict={'family': 'serif','weight': 'bold',\
                                                   'size': 16,'color':'darkred'})
plt.ylabel('No of samples tested( 1 unit=1 Million)',fontsize=18)
plt.xlabel('Date',fontsize=18)
plt.show()
plt.tight_layout()

## _Test Positivity Rate_

In [ ]:
import matplotlib.dates as md
plt.figure(figsize=(20,15))
bar = sns.barplot(data=icmr[~(icmr['Test positivity rate']==0)][::3],x='Tested As Of',y='Test positivity rate')
bar.set_xticklabels(bar.get_xticklabels(), rotation=45, horizontalalignment='right',fontsize=10.5)
plt.title('Times Series plot for Test Postivity rate',fontdict={'family': 'serif','weight': 'bold',\
                                                   'size': 16,'color':'black'})
plt.ylabel('Test Postivity rate',fontsize=15)
plt.xlabel('Date',fontsize=15)
plt.show()
plt.tight_layout()

#### <font color=darkgreen>_Notice Test Positivity Rate on 29-10-2020_

#### _Test Positive rate describe for better understanding_

In [ ]:
icmr[~(icmr['Test positivity rate']==0)]['Test positivity rate'].describe()

## _Test Per Million_

In [ ]:
import matplotlib.dates as md
plt.figure(figsize=(20,15))
bar = sns.barplot(data=icmr[~(icmr['Tests per million']==0)],x='Tested As Of',y='Tests per million')
bar.set_xticklabels(bar.get_xticklabels(), rotation=45, horizontalalignment='right')
plt.title('Times Series plot for Test per million',fontdict={'family': 'serif','weight': 'bold',\
                                                   'size': 16,'color':'black'})
plt.ylabel('Test per million',fontsize=15)
plt.xlabel('Data from March 13, 2020 till Oct 29, 2020 ',fontsize=15)
plt.tick_params(labelbottom=False)
plt.show()
plt.tight_layout()

#### <font color=darkgreen> Test per million are increasing at exponential rate as date increases_

### _Test per million Describe for better understanding_

In [ ]:
icmr['Tests per million'].describe()

## _Total Positive Case_

In [ ]:
icmr['Total Positive Cases']

In [ ]:
import matplotlib.dates as md
plt.figure(figsize=(20,15))
bar = sns.barplot(data=icmr[~(icmr['Total Positive Cases']==0)][::3],x='Tested As Of',y='Total Positive Cases')
bar.set_xticklabels(bar.get_xticklabels(), rotation=45, horizontalalignment='right',fontsize=10.5)
plt.title('Times Series plot for Total Positive cases',fontdict={'family': 'serif','weight': 'bold',\
                                                   'size': 16,'color':'black'})
plt.ylabel('Total Positive Cases',fontsize=15)
plt.xlabel('Date',fontsize=15)
plt.show()
plt.tight_layout()

In [ ]:
icmr[icmr['Total Positive Cases']!=0]['Total Positive Cases'].describe()

## _Relation between Test Positivity rate and Tests per million_

In [ ]:
tpr = icmr[icmr['Test positivity rate']!=0]
tpr.head()

In [ ]:

plt.subplots(figsize = (8,6))
mask = np.zeros_like(tpr.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
#Plotting heatmap
sns.heatmap(tpr.corr(), cmap=sns.diverging_palette(20, 220, n=200), mask = mask, annot=True, center = 0) # add _r for reverse color map

#### <font color=darkgreen>_Correlation cofficients_

In [ ]:
tpr['Test positivity rate'].corr(tpr['Tests per million'])

In [ ]:
plt.figure(figsize=(10,8))
plt.plot(tpr['Total Samples Tested'],tpr['Total Positive Cases'],c='darkblue',lw=3)
plt.xlabel('Total Samples Tested',fontsize=14)
plt.ylabel('Total Positive Cases',fontsize=14)
plt.title('Rlationship between Total Sampleas tested and Total Posititve Cases',fontdict={'family': 'serif','weight': 'bold',\
                                                   'size': 14})
plt.show()

#### <font color=darkgreen>_Total positive cases increasing at the same rate as Total Sample tested, we can see approximate straight line of 45 degree slope. If testing increases in india positive cases will increase at the same rate.As you can see no of positive cases also increases(pearson correlation cofficient is 1_</font>

In [ ]:
import matplotlib.dates as md
plt.figure(figsize=(12,8))
bar = sns.barplot(data=icmr[~(icmr['Tests Per Confirmed Case']==0)],x='Tested As Of',y='Tests Per Confirmed Case')
bar.set_xticklabels(bar.get_xticklabels(), rotation=45, horizontalalignment='right',fontsize=12)
plt.title('Times Series plot for Tests Per Confirmed Case',fontdict={'family': 'serif','weight': 'bold',\
                                                   'size': 16,'color':'black'})
plt.ylabel('Tests Per Confirmed Case',fontsize=15)
plt.xlabel('Date',fontsize=15)
plt.show()
plt.tight_layout()

#### <font color=darkgreen>_Test per confirmed cases saturated after April 4, 2020, because testing samples increases at a fast rate in april and after that, avearge 1 lakh samples daily tested in July.</font>

## _StateWise Tested numbers Data_

In [ ]:
state_w= pd.read_csv('../input/state-wise-dataset/State/statewise_tested_numbers_data.csv')
state_w2= pd.read_csv('../input/state-wise-dataset/State/statewise_tested_numbers_data.csv')

In [ ]:
state_w.info()

In [ ]:
state_w.columns

In [ ]:
state_w=state_w[['Updated On', 'State', 'Total Tested', 'Positive',
       'Negative', 'Total Num Beds (Normal/Isolation) ', 'Total Num ICU Beds',
       'Total Num Ventilators', 'Test positivity rate', 'Tests per thousand',
       'Tests per million', 'Tests per positive case','Total N95 Masks','Total PPE',
       'Population NCP 2019 Projection']]
state_w.head()

In [ ]:
plt.figure(figsize=(18,10))
mask = np.zeros_like(state_w[state_w.columns[2:-1]].corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(state_w[state_w.columns[2:-1]].corr(), cmap=sns.diverging_palette(20, 220, n=200), mask = mask, annot=True, center = 0)
#sns.heatmap(state_w[state_w.columns[2:-1]].corr(),cmap='viridis_r',annot=True,annot_kws={"size":13,'weight':'bold'})
plt.show()

#### <font color=darkgreen>_Correlation cofficient between different features, how correlated are they with each other_

In [ ]:
state_w['Updated On']=pd.to_datetime(state_w['Updated On'],format='%d/%m/%y').sort_values()

In [ ]:
#state_w['Updated On']=state_w['Updated On'].astype('object').astype('str').apply(lambda x:x.split(' ')[0])

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(20,15))
cmapp = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'pink','orchid','khaki','ivory','gold','silver','sienna','silver',\
        'turquoise','wheat','yellowgreen','magenta','azure','orangered','plum','indigo','goldenrod','lime','crimson',\
        'lavender','salmon','tomato','coral','grey','tan','darkblue','aqua','chocolate']
sns.lineplot(data=state_w,x='Updated On',y='Total Tested',hue='State',palette=cmapp,lw=2.5)
plt.title('Total Test Time Series Data State Wise',fontdict={'family': 'serif','weight': 'bold','size': 14})
plt.xlabel('Date',fontsize=15)
plt.ylabel('Total Test',fontsize=15)

plt.show()

#### <font color=darkgreen>You can notice that after September , Uttar Pradesh increase rate is maximum. Sikkim , Meghalaya, Arunachal pradesh have the lowest rate or gradient.
    

## _StateWise Daily Cases Time Series_

In [ ]:
sw_dc =pd.read_csv('../input/state-wise-dataset/State/state_wise_daily.csv')
sw_dc.drop('Date_YMD',axis=1,inplace=True)
sw_dc=sw_dc.melt(id_vars=['Date','Status'])
sw_dc.rename(columns={'variable':'StateCode'},inplace=True)
sw_dc.head()

In [ ]:
x = sw_dc[sw_dc['Status']=='Confirmed'].groupby('StateCode')['value'].sum().sort_values(ascending=False).head(11)
x.index


In [ ]:
sw_dc1 = sw_dc[sw_dc['StateCode'].isin(['MH', 'TN', 'DL', 'GJ', 'KA', 'UP', 'TG','WB', 'AP', 'RJ'])]
sw_dc1.head()

In [ ]:
sw_dc1['Date']=pd.to_datetime(sw_dc1['Date'],format='%d-%b-%y').sort_values()
sw_dc2=sw_dc1.copy()
sw_dc2= sw_dc2[sw_dc2['Status']=='Confirmed']

## Rate of Increase of cases in top 10 state case wise w.r.t date

In [ ]:
sw_dc2['value']=sw_dc2.groupby('StateCode')['value'].cumsum()
sw_dc2.head()

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(14,10))
cmapp = ['b', 'g', 'r', 'm', 'y', 'k', 'pink','orange','yellowgreen','chocolate']
sns.lineplot(x=sw_dc2['Date'],y=sw_dc2['value'],hue=sw_dc2['StateCode'],palette=cmapp,lw=4)
plt.title('Top10 state cases with time series',fontdict={'family': 'serif','weight': 'bold','size': 14})
plt.xlabel('Date',fontsize=15)
plt.ylabel('Total Sample tested',fontsize=15)
plt.legend(fancybox=True,frameon=True,edgecolor='r',facecolor='g',framealpha=0.2)
plt.show()

#### <font color=darkgreen>Maharashtra  has the highest increasing rate, followed by Andra Pradesh and Karnantaka. Note: Cases rate for delhi is changing unevenly.Maharashtra highest slope value. Hence, Cases in Maharashtra is highest followed by Tamil Nadu.

In [ ]:
tpm = pd.DataFrame(state_w.groupby('State')['Tests per million'].max())
tpm.head(2)

In [ ]:
tpt = pd.DataFrame(state_w.groupby('State')['Tests per thousand'].max())
tpt.head(2)

In [ ]:

test_pr = pd.DataFrame(state_w.groupby('State')['Test positivity rate'].last())
test_pr.head(2)


In [ ]:
pop = pd.DataFrame(state_w.groupby('State')['Population NCP 2019 Projection'].first())
pop.head(2)


In [ ]:
hdi = pd.read_excel('../input/state-wise-dataset/State/HDI_India.xlsx')
hdi.rename(columns={'State/Union Territory':'State'},inplace=True)
hdi.set_index('State',inplace=True)
hdi.head(2)


In [ ]:
states_t= pd.read_csv('../input/state-wise-dataset/State/state_wise.csv')
states_t.set_index('State',inplace=True)
states_t.head(2)

In [ ]:
print(len(hdi))
print(len(states_t))
print(len(pop))
print(states_t.index)
print(pop.index)
print(set(pop.index) - set(states_t.index))

In [ ]:
states_t = pd.concat([states_t,tpm],axis=1)
states_t = pd.concat([states_t,tpt],axis=1)
states_t= pd.concat([states_t,test_pr],axis=1)
states_t= pd.concat([states_t,pop],axis=1)
states_t= pd.concat([states_t,hdi],axis=1)

In [ ]:
print(states_t.index)
print(states_t.columns)
print(len(states_t))
print(len(states_t.columns))


In [ ]:
states_t.rename(columns={'Population NCP 2019 Projection':'Population'},inplace=True)
states_t.drop(labels=['Last_Updated_Time','Migrated_Other','Delta_Confirmed','Delta_Recovered','Delta_Deaths',\
                   'State_Notes'],inplace=True,axis=1)
states_t.head(2)

In [ ]:
states= states_t[1:]
states['Deaths per million']= (states['Deaths']/states['Population'])*1000000
states['Num of persons per allopathic Doctor']= states['Population']/states['Num of Allopathic Doctors']
states['Num of persons per hospital']= states['Population']/states['Tot num of hospitals']
states['Num of persons per hospital bed']= states['Population']/states['Tot num of hospital beds']
states['Num of persons per icu bed']= states['Population']/states['Tot num of ICU beds']
states['Num of persons per ventilator beds']= states['Population']/states['Tot num of ventilator beds']
states['recovery percentage']= states['Recovered']/states['Confirmed']*100
states['death percentage']= states['Deaths']/states['Confirmed']*100


In [ ]:

states['Confirmed']= pd.to_numeric(states['Confirmed'])
states.info()

In [ ]:
states.head(2)

## Total Num of cases per state

In [ ]:
states.head()

In [ ]:
sns.set_style('white')

plt.figure(figsize=(18,15))
ax = plt.barh(y=states.index,width=states.Confirmed,color='red',height=0.9)
plt.title('Total Num of cases per State',fontdict={'family': 'serif','color':'red','weight': 'heavy',\
                                                   'size': 14})
plt.ylabel('States',fontsize=14)
plt.xlabel('Total num of Cases',fontsize=14)
# remove all the ticks (both axes), and tick labels on the Y axis
plt.tick_params(axis='both',top=False, bottom=False, left=False, right=False, labelleft=True, labelbottom=False)

for p in ax.patches:
    width = p.get_width()
    plt.text(width+42000,p.get_y()+p.get_height()/2.5,'{:1.1f}'.format(width/1000)+'k',ha="center",fontsize=11,fontweight='bold',\
            alpha=0.8) 
    
    # remove the frame of the chart
plt.show()

#### <font color=darkgreen>_Total no. of cases in Maharashtra has highest followed by Andra Pradesh. while Lakshadweep  and Mizoram has lowest no of cases._

## Total num of people deceased per state

In [ ]:
sns.set_style('white')
plt.figure(figsize=(18,14))
ax = plt.barh(y=states.index,width=states.Deaths,color='blue',height=1)
plt.title('Num of People deceased per State',fontdict={'family': 'serif','color':'blue','weight': 'heavy',\
                                                   'size': 14})
plt.ylabel('States',fontsize=14)
plt.xlabel('Num of People deceased',fontsize=14)
# remove all the ticks (both axes), and tick labels on the Y axis
plt.tick_params(axis='both',top=False, bottom=False, left=False, right=False, labelleft=True, labelbottom=False)

for p in ax.patches:
    width = p.get_width()
    plt.text(width+1000,p.get_y()+p.get_height()/4,'{:1.0f}'.format(width),ha="center",fontsize=11,fontweight='bold',\
            alpha=0.8) 
    
    # remove the frame of the chart
plt.show()

## Total no of patients confirmed,Recovered,Deceased per state

In [ ]:
df1=states[['Recovered','Deaths','Confirmed']].reset_index()
df1.rename(columns={'index':'State'},inplace=True)
df1.head()

In [ ]:
import plotly
import cufflinks as cf
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
np.random.seed(123)
cf.set_config_file(theme='henanigans')
#layout1 = cf.Layout(height=300,width=200)

fig = df1.iplot(asFigure=True,kind='bar',barmode='stack',x='State',yTitle='Total no of patients confirmed,\
Recovered,Deceased',xTitle='States',title='Total no of patients confirmed,Recovered,Deceased per state',colors=['orange','blue','green'])
fig.update_layout(width=1000,height=1000)
#legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
iplot(fig)

#### <font color=darkgreen>_Maharashtra has highest death, recoveries and total cases._

In [ ]:
cf.tools.getThemes()

In [ ]:
import plotly
import cufflinks as cf
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#layout1 = cf.Layout(height=300,width=200)
init_notebook_mode(connected=True)
#np.random.seed(123)
cf.set_config_file(theme='henanigans')
fig = states['Tests per million'].reset_index().iplot(asFigure=True,kind='bar',x='index',xTitle='States',\
                                                      yTitle='Test per million',title='Test per million per state',colors=['green'])
fig.update_layout(width=1000,height=1000)
#legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
iplot(fig)

#### <font color=darkgreen>_Ladakh has highest test per million followed by Ladakh,while Madhya Pradesh has lowest test per million.

In [ ]:
import plotly
import cufflinks as cf
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
#layout1 = cf.Layout(height=300,width=200)
init_notebook_mode(connected=True)
#np.random.seed(123)
cf.set_config_file(theme='henanigans')
fig = states['Deaths per million'].reset_index().iplot(asFigure=True,kind='bar',x='index',xTitle='States',\
                                                      yTitle='Deaths per million',title='Deaths per million per state',colors=['red'])
#fig.update_layout(width=1000,height=1000,xaxis=dict(tickfont=dict(color="white")))
fig.update_layout(width=1000,height=1000)

iplot(fig)
                  
                  

#### <font color=darkgreen>_Puducherry has highest deaths per million followed by Goa._

## Test Positivity Ratio and Recovery Percentage per state

In [ ]:
states['Test positivity rate']=states['Test positivity rate'].astype('str').apply(lambda x:x.split('%')[0])
df2=states[['Test positivity rate','recovery percentage']]
df2.reset_index(inplace=True)
df2.rename(columns={'index':'State'},inplace=True)
df2.head()

In [ ]:
import plotly
import cufflinks as cf
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
np.random.seed(123)
cf.set_config_file(theme='henanigans')
#layout1 = cf.Layout(height=300,width=200)

fig = df2.iplot(asFigure=True,kind='bar',barmode='stack',x='State',yTitle='Test Positivity Ratio and Recovery Percentage',xTitle='States',title='Test Positivity Ratio and Recovery Percentage per state',colors=['orange','green'])
fig.update_layout(width=1000,height=1000)
#legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
iplot(fig)

#### <font color=darkgreen>_Maharashtra has highest recovery rate followed by Chandigarh, while Arunachal Pradesh has lowest recovery rate._

In [ ]:


states.reset_index(inplace=True)
states.rename(columns={'index':'State'},inplace=True)
states.head(2)

## HDI per state

In [ ]:
import plotly
import cufflinks as cf
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
np.random.seed(123)
cf.set_config_file(theme='henanigans')
#layout1 = cf.Layout(height=300,width=200)

fig = states[['State','HDI ']].iplot(asFigure=True,kind='bar',x='State',yTitle='HDI Index',xTitle='States',title='HDI Index per state',colors=['green'])
fig.update_layout(width=1000,height=1000)
fig.add_shape(type="line",xref="paper",yref='paper',x0=0,y0=states['HDI '].median(),x1=1,y1=states['HDI '].median(),line=dict(color="DarkOrange",width=3))
#legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
iplot(fig)


#### <font color=darkgreen>_Kerela has highest HDI followed by Chandigarh while Bihar has lowest HDI._

In [ ]:
states['Num of persons per allopathic Doctor'].mean()

In [ ]:
import plotly
import cufflinks as cf
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
np.random.seed(123)
cf.set_config_file(theme='henanigans')
#layout1 = cf.Layout(height=300,width=200)

fig = states[['State','Num of persons per allopathic Doctor']].iplot(asFigure=True,kind='bar',x='State',yTitle='Num of persons per allopathic Doctor',xTitle='States',title='Num of persons per allopathic Doctor per state',colors=['green'])
fig.update_layout(width=1000,height=1000)
fig.add_shape(type="line",x0=0,y0=states['Num of persons per allopathic Doctor'].mean(),x1=37,y1=states['Num of persons per allopathic Doctor'].mean(),line=dict(color="DarkOrange",width=3))

#legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
iplot(fig)


#### <font color=darkgreen>_Goa has the lowest Num of persons per allopathic Doctor, Nagalanad and Mizoram has highest. Goa is best state accoridng to Num of persons per allopathic Doctor._

## Total num of hospital beds in each state

In [ ]:
hosp_beds = state_w2.groupby('State')['Total Num Beds (Normal/Isolation) '].max()
hosp_beds = pd.DataFrame(hosp_beds).reset_index()


In [ ]:
import plotly
import cufflinks as cf
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
np.random.seed(123)
cf.set_config_file(theme='henanigans')
#layout1 = cf.Layout(height=300,width=200)

fig = hosp_beds[['State','Total Num Beds (Normal/Isolation) ']].iplot(asFigure=True,kind='bar',\
                                                   barmode='stack',x='State',yTitle='Total Num Beds (Normal/Isolation) ',xTitle='States',title='Total Num Beds (Normal/Isolation) in each state',colors=['olive'])
fig.update_layout(width=1000,height=1000)
#legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
iplot(fig)

#### <font color=darkgreen>_Odisha has highest no hospital beds followed by Karnataka whereas Manipur has lowest beds._

## Total num of ICU and ventilators in diffferent states

In [ ]:
state_w2.columns

In [ ]:
icu_beds = state_w2.groupby('State')['Total Num ICU Beds'].max()
icu_beds = pd.DataFrame(icu_beds).reset_index()

In [ ]:
import plotly
import cufflinks as cf
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
np.random.seed(123)
cf.set_config_file(theme='henanigans')
#layout1 = cf.Layout(height=300,width=200)

fig = states[['State','Tot num of ICU beds','Tot num of ventilator beds']].iplot(asFigure=True,kind='bar',\
                                                                                 barmode='stack',x='State',yTitle='Total num of ICU and ventilators',xTitle='States',title='Total num of ICU and ventilators in diffferent states',colors=['green','orange'])
fig.update_layout(width=1000,height=1000)
#legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
iplot(fig)

#### <font color=darkgreen>_UP has highest no of vemtilators and ICU beds followed by Karnataka whereas Manipur has lowest numbers._

## Number of persons per icu bed and  ventilator bed in states

In [ ]:
import plotly
import cufflinks as cf
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
np.random.seed(123)
cf.set_config_file(theme='henanigans')
#layout1 = cf.Layout(height=300,width=200)

fig = states[['State','Num of persons per icu bed','Num of persons per ventilator beds']].iplot(asFigure=True,kind='bar',\
                                                                                 barmode='stack',x='State',yTitle='Number of persons per icu and ventilator bed',xTitle='States',\
                                                                                                title='Number of persons per icu and ventilator bed in states',colors=['green','orange'])
fig.update_layout(width=1000,height=1000)
#legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
iplot(fig)

#### <font color=darkgreen>_Bihar has highest number of person per icu and ventilator bed (i.e. Bihar has worst healthcare compared to other states), whereas Chandigarh has lowest num of person per icu and ventilator(i.e. best healthcare compared to other states)_

## Num of persons per hospital bed for different states

In [ ]:
import plotly
import cufflinks as cf
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
np.random.seed(123)
cf.set_config_file(theme='henanigans')
#layout1 = cf.Layout(height=300,width=200)

fig = states[['State','Num of persons per hospital bed']].iplot(asFigure=True,kind='bar',\
                                                                                 barmode='stack',x='State',yTitle='Num of persons per hospital bed',xTitle='States',\
                                                                                                title='Num of persons per hospital bed for different states',colors=['olive'])
fig.update_layout(width=1000,height=1000)
#legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
iplot(fig)

## Death Percentage for different states

In [ ]:
import plotly
import cufflinks as cf
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
np.random.seed(123)
cf.set_config_file(theme='henanigans')
#layout1 = cf.Layout(height=300,width=200)

fig = states[['State','death percentage']].iplot(asFigure=True,kind='bar',\
                                                                                 barmode='stack',x='State',yTitle='death percentage',xTitle='States',\
                                                                                                title='Death percentage for different states',colors=['red'])
fig.update_layout(width=1000,height=1000)
#legend=dict(yanchor="top",y=0.99,xanchor="left",x=0.01)
iplot(fig)

#### <font color=darkgreen>_Punjab has the highest death percentage followed by maharashtra whereas Lakashwadeep has lowest._

In [ ]:
states['Test positivity rate']=states['Test positivity rate'].astype('float32')

In [ ]:
df_htmap= states[['Confirmed', 'Recovered', 'Deaths', 'Active',
       'Tests per million', 'Tests per thousand', 'Test positivity rate',
       'Population', 'HDI ', 'Tot num of hospitals',
       'Tot num of hospital beds', 'Tot num of ICU beds',
       'Tot num of ventilator beds', 'Pop Density per Km2',
       'Num of Allopathic Doctors', 'Deaths per million',
       'Num of persons per allopathic Doctor', 'Num of persons per hospital',
       'Num of persons per hospital bed', 'Num of persons per icu bed',
       'Num of persons per ventilator beds', 'recovery percentage',
       'death percentage']]
plt.subplots(figsize = (30,20))
mask = np.zeros_like(df_htmap.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
#Plotting heatmap
sns.heatmap(df_htmap.corr(), cmap=sns.diverging_palette(20, 220, n=200), mask = mask, annot=True, center = 0)

In [ ]:
sns.set()
plt.figure(figsize=(15,10))
sns.lineplot(data=states,x='Confirmed',y='death percentage',color='r',lw=3)
sns.lineplot(data=states,x='Confirmed',y='recovery percentage',color='b',lw=2.5)
sns.lineplot(data=states,x='Confirmed',y='Test positivity rate',color='g',lw=3)
plt.legend(['Death Percentage','Recovery Percentage','Test positivity rate'])
plt.xlabel('No of Confirmed Cases')
plt.show()

# Forecasting using FBProphet

In [ ]:
stats[stats['State']=='Maharashtra'][['Date','Confirmed']].tail()

# Maharashtra Confirmed Cases Forecast

In [ ]:
stats[stats['State']=='Maharashtra'][['Date','Confirmed']].tail(10)

In [ ]:
df_phet_MH = stats[stats['State']=='Maharashtra'][['Date','Confirmed']]

In [ ]:
df_phet_MH= df_phet_MH[df_phet_MH['Date']<'2020-10-20']
fb_pro = Prophet(yearly_seasonality=True,daily_seasonality=True)
df_phet_MH.columns = ['ds' , 'y']

In [ ]:
fb_pro.fit(df_phet_MH)

In [ ]:
df_phet_future = fb_pro.make_future_dataframe(periods= 20) 
df_phet_future.tail()

In [ ]:
forecast=fb_pro.predict(df_phet_future)
forecast.head()

In [ ]:
fig = plot_plotly(fb_pro, forecast)
fig.update_layout(
                  autosize=False,
                  width= 750,
                  height= 800,
    title_text='Covid-19 Forecast',
    title_x=0.5,
    paper_bgcolor='mistyrose',
    plot_bgcolor = 'mistyrose',)

fig.show()

# Delhi Confirmed Cases Forecast

In [ ]:
stats[stats['State']=='Delhi'][['Date','Confirmed']].tail(10)

In [ ]:
df_phet_DL = stats[stats['State']=='Delhi'][['Date','Confirmed']]
df_phet_DL= df_phet_DL[df_phet_DL['Date']<'2020-10-20']
fb_pro = Prophet(yearly_seasonality=True,daily_seasonality=True,weekly_seasonality=True)
df_phet_DL.columns = ['ds' , 'y']

In [ ]:
fb_pro.fit(df_phet_DL)
df_phet_future = fb_pro.make_future_dataframe(periods= 20)
forecast=fb_pro.predict(df_phet_future)

In [ ]:
fig = plot_plotly(fb_pro, forecast)
fig.update_layout(
                  autosize=False,
                  width= 750,
                  height= 800,
    title_text='Covid-19 Forecast',
    title_x=0.5,
    paper_bgcolor='tan',
    plot_bgcolor = 'tan',)

fig.show()

# Maharashtra Deceased Cases Forecast

In [ ]:
stats[stats['State']=='Maharashtra'][['Date','Deceased']].tail(10)

In [ ]:
df_phet_MH = stats[stats['State']=='Maharashtra'][['Date','Deceased']]
df_phet_MH= df_phet_MH[df_phet_MH['Date']<'2020-10-20']
fb_pro = Prophet(yearly_seasonality=True,daily_seasonality=True)
df_phet_MH.columns = ['ds' , 'y']

In [ ]:
fb_pro.fit(df_phet_MH)
df_phet_future = fb_pro.make_future_dataframe(periods= 20) 
df_phet_future.tail()
forecast=fb_pro.predict(df_phet_future)

In [ ]:
fig = plot_plotly(fb_pro, forecast)
fig.update_layout(
                  autosize=False,
                  width= 750,
                  height= 800,
    title_text='Covid-19 Forecast',
    title_x=0.5,
    paper_bgcolor='khaki',
    plot_bgcolor = 'khaki',)

fig.show()

# Delhi Deceased Cases Forecast

In [ ]:
stats[stats['State']=='Delhi'][['Date','Deceased']].tail(10)

In [ ]:
df_phet_DL = stats[stats['State']=='Maharashtra'][['Date','Deceased']]
df_phet_DL= df_phet_DL[df_phet_DL['Date']<'2020-10-20']
fb_pro = Prophet(yearly_seasonality=True,daily_seasonality=True)
df_phet_DL.columns = ['ds' , 'y']

In [ ]:
fb_pro.fit(df_phet_DL)
df_phet_future = fb_pro.make_future_dataframe(periods= 20) 
df_phet_future.tail()
forecast=fb_pro.predict(df_phet_future)

In [ ]:
fig = plot_plotly(fb_pro, forecast)
fig.update_layout(
                  autosize=False,
                  width= 750,
                  height= 800,
    title_text='Covid-19 Forecast',
    title_x=0.5,
    paper_bgcolor='yellowgreen',
    plot_bgcolor = 'yellowgreen',)

fig.show()

# Forecasting using  ARIMA model

# Maharashtra Confirmed Cases Forecast

In [ ]:
sns.set_style('whitegrid')
from statsmodels.tsa.arima_model import ARIMA
arima = ARIMA(df_phet_MH['y'], order=(5, 1, 0))
arima = arima.fit(trend='c', full_output=True, disp=True)
forecast = arima.forecast(steps= 30)
pred = list(forecast[0])

start_date = df_phet_MH['ds'].max()
prediction_dates = []
for i in range(30):
    date = start_date + datetime.timedelta(days=1)
    prediction_dates.append(date)
    start_date = date
plt.figure(figsize= (15,10))
plt.xlabel("Dates",fontsize = 20)
plt.ylabel('Total Confirmed cases for Maharahstra',fontsize = 20)
plt.title("Predicted Values for the next 30 Days" , fontsize = 20)

plt.plot_date(y= pred,x= prediction_dates,linestyle ='dashed',color = 'red',label = 'Predicted');
plt.plot_date(y=df_phet_MH['y'],x=df_phet_MH['ds'],linestyle = '-',color = 'blue',label = 'Actual');
plt.legend();

# Delhi Confirmed Cases Forecast

In [ ]:
sns.set_style('whitegrid')
from statsmodels.tsa.arima_model import ARIMA
arima = ARIMA(df_phet_DL['y'], order=(5, 1, 0))
arima = arima.fit(trend='c', full_output=True, disp=True)
forecast = arima.forecast(steps= 30)
pred = list(forecast[0])

start_date = df_phet_DL['ds'].max()
prediction_dates = []
for i in range(30):
    date = start_date + datetime.timedelta(days=1)
    prediction_dates.append(date)
    start_date = date
plt.figure(figsize= (15,10))
plt.xlabel("Dates",fontsize = 20)
plt.ylabel('Total Confirmed cases for Delhi',fontsize = 20)
plt.title("Predicted Values for the next 1 month" , fontsize = 20)

plt.plot_date(y= pred,x= prediction_dates,linestyle ='dashed',color = 'red',label = 'Predicted');
plt.plot_date(y=df_phet_DL['y'],x=df_phet_DL['ds'],linestyle = '-',color = 'blue',label = 'Actual');
plt.legend();

![](https://media.giphy.com/media/PnD1HSO0rVx8AiNIMl/giphy.gif)

# <font color=darkred>`If you found this notebook helpful , some upvotes would be very much appreciated - That will keep me motivated :)`


# <----------------------THANK YOU :)---------------------->